<a href="https://colab.research.google.com/github/salarbalou/steadystate_analysis/blob/main/plot_photothermal_Tvst_profile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import numpy as np
import requests as req
import plotly.io as pio
import plotly.graph_objects as go
import re
import sympy as sy
from sympy import symbols, diff
from sympy.solvers import solve
from scipy.misc import derivative
import string as sr
pio.templates.default ='plotly_dark'
from plotly.subplots import make_subplots
import plotly.express as px

In [2]:
def findmb(x, y): # a simple function to find the slope and intecept of a line
  m,b = np.polyfit(x, y, deg= 1)
  return m,b

In [61]:
#date_hint = '10-19-21'
#compound_hint = 'GNR , GNS , CQD'
#concentration_hint = '10x, 5x, 1.5x'
#duty_cycle_hint = 'dc20' , 'dc30'
dcs= ['dc20', 'dc30', 'dc40', 'dc50', 'dc60', 'dc70', 'dc80', 'dc90'] # a list of duty cycles available
all_equ_temp = [] # an empty list of equ_temp that can be used to finally store equ_temps in it
all_laser_power_density = [] # an empty list of laser_power_density that can bes used to store the values of LPS generated in the for loop
all_eta = [] # aan empty list of eta values (photothermal conversion efficiency) that can bes used to store the values of LPS generated in the for loop
for dc in dcs: # since we have multiple dutycylce values we make a for loop
 baseurl = 'https://raw.githubusercontent.com/salarbalou/steadystate_analysis/main/' # base url of the text file 
 char = '.txt' # format of the file
 compound_and_concentration_dutycycle= '10-21-21-CQD-10x-'+dc #experiment was performed on 10-20-21 using CQD 10x/ dutycycle values change
 text = req.get(baseurl+compound_and_concentration_dutycycle+char).text # we download and store the text of each file.txt
 data_path = re.compile(r'^([0-9.]+)[\s,]+([0-9.]+)+[\s,]+([-0-9.]+)+[\s,]+([0-9]+)', re.MULTILINE) # we grab the desired text within the text file (all data except for space and comma)
 data_list = data_path.findall(text) # this is a list of quadruple strings: each quadruple in this list have abcd string in it. 
 data = []
 for quadruple in data_list:
      eachquadruple = [] # this list here stores the float version of abcd in each quadruple: eachquadruple is a list of four float values
      for abcd in quadruple:
           eachquadruple.append(float(abcd))
      data.append((eachquadruple[0], eachquadruple[1], eachquadruple[2], eachquadruple[3])) # we append all eachquadruples in data
 final_data = np.array(data) # here we make an array of the list: final_data is a matrix with 183 rows and 4 columns (t, T, I, dc)
 time = final_data[:, 0][1:162]
 temperature = final_data[:, 1][1:162]
 temperature1 = temperature[22:]
 equ_temp = np.max(temperature, axis=0) # as mentioned, we assume the max T is the equ_T
 all_equ_temp.append(equ_temp)
 current = np.sum(-final_data[:, 2][24:162], axis = 0)/138 #Amps
 dutycycle = final_data[:, 3][23:162]
 beam_height = 0.1 #cm
 beam_width = 0.1 #cm
 voltage_value = np.linspace(6, 7.5, num = 230) # max voltage is 7.5V, min voltage required for laser to turn on is 6V. if we polyfit V with dutycyle(0:250), find the relation of V and dc
 dutycycle_value  = np.arange(20, 250,1)
 m, b = findmb(dutycycle_value, voltage_value) 
 realvoltage = np.sum(m*dutycycle+b)/dutycycle.size #V
 laser_power_density = np.array(((realvoltage*current)/(beam_height*beam_width))) #W/cm^2
 all_laser_power_density.append(laser_power_density)
 dt = np.gradient(time)
 dT = np.gradient(temperature)
 diffTt = np.sqrt((dT/dt)**2)
 fig = make_subplots(rows = 1, cols = 2, subplot_titles=('Temperature vs time at '+dc, 'dT/dt vs time at '+dc))
 fig.add_scatter(x = time, y = temperature, mode = 'markers+lines', marker_color = 'red', row = 1, col = 1, name = 'Temperature vs time')
 fig.add_scatter(x = time, y = diffTt, mode = 'markers+lines', marker_color ='red' , line_color = "green", row  = 1, col  =2, name='dT/dt vs time')
 fig.update_xaxes(title_text = 'time (seconds)', row = 1, col = 1)
 fig.update_xaxes(title_text = 'time (seconds)', row = 1, col = 2)
 fig.update_yaxes(title_text = 'Temperature', row = 1, col = 1)
 fig.update_yaxes(title_text = 'dT/dt', row = 1, col = 2)
 fig.update_layout(height = 800, width = 2400)
 fig.show()

 #I0 = #incident laser power W
 #Itr = #transmitted laser power W
 #eta = # photothermal conversion eficiency
 #h = #heat transfor coefficient between solution and environment+ W/mk
 #S = #surface area seperating solution from environemnt +m^2
 #Tw = #solution temp + K
 #T0 = #environment temp + K
 #m = #mass of solition + gr
 #Cp = #specific heat capacity of solution + J/g.K
 #v= volume of the CQD used in microliter + m^3
 #R = heating rate +
 r = symbols('r')
 I0, Itr, eta, h, S, Tw, T0, m, Cp, R = symbols('I0, Itr, eta, h, S, Tw, T0, m, Cp, R')
 I0 = current*realvoltage #W

 Itr = (current*realvoltage*0.75) # needs to be experimentally calculated ##NOT YET DONE #W


 T0 = temperature[0]+273.15 #K
 Tw = equ_temp+273.15 #K
 m= 1e-2 #gr based on density of water to be 1g/cm^3
 R = 1.5*1e-3 #radius of the volume filled with CQDs for 10ul CQD #m
 H  = 3.2*1e-3 # height of the volume filled with CQDs for 10ul CQD #m
 hmin = 598*1e-3 # W/mK at 20C
 hmax = 677*1e-3 # W/mK at 100
 mh, bh = findmb(np.arange(20, 100, 1)+273, np.arange(hmin, hmax, 1e-3))
 h = np.max((mh*(temperature[22:112]+273.15)+bh))  #W/mk
 S = np.pi*R*(R+np.sqrt((H)**2+(R)**2)) #Surface Area of cone filled with CQD #m^2
 R = findmb(np.arange(np.sort(temperature[22:112]).size), temperature[22:112]+273.15)[0] #K/s
 Cp = 4.184 #J/g.K for water
 eta = (R*(Cp*m)+np.sum((h*S*(Tw - T0))))/(I0-Itr) # Equation to find the photothermal conversion efficiency
 all_eta.append(eta)


if (np.array(all_equ_temp).size>1): # this will show the Temperature vs DutyCycle plot only in case if we have more than 1 utilized dutycycle
 fig2 = make_subplots(specs=[[{"secondary_y": True}]])
 fig2.add_trace(go.Scatter(x = all_laser_power_density, y= all_equ_temp, mode = 'markers+lines',name = 'Temperature vs Laser Power Density', marker_color = 'red'), secondary_y = False)
 fig2.add_trace(go.Scatter(x = all_laser_power_density, y= all_eta, mode = 'markers+lines', name = 'Photothermal Converson Effeciency vs Laser Power Density', marker_color = 'blue'), secondary_y = True)
 fig2.update_layout(height = 800, width= 2400)
 fig2.update_xaxes(title = "Laser Power Density (W/cm^2)")
 fig2.update_yaxes(title = 'Temperature', secondary_y=False)
 fig2.update_yaxes(title = 'Photothermal Conversion Efficiency', secondary_y=True)
 fig2.show()
else:
 print('No Results')

In [6]:
m,b=np.polyfit(np.arange(20, 100, 10), np.array(all_equ_temp), deg=1) # this function 
def givemedc(T):
  req_dc = (T-b)/m
  return np.int(req_dc)
givemedc(65) 
# Please note that this function only return the required DutyCylce for equ T for CQD of 10x concentration,
# in case of increase in concentration this value must increase (10 points for halfed-experimentally calculated)

45

In [62]:
def laplacian (f, x, y, z):
 return diff(f(x), x , 2)+diff(f(x), y , 2)+diff(f(x), z , 2)